
# FLEET MANAGEMENT SETUP

### CREATE A DATABASE FOR THE FLEET MANAGEMENT SOLUTION

In [ ]:
CREATE DATABASE IF NOT EXISTS FLEET_MANAGEMENT;

In [ ]:
CREATE SCHEMA IF NOT EXISTS FLEET_MANAGEMENT.ANALYTICS

## SOURCE TABLES

In [ ]:
CREATE OR REPLACE TABLE FLEET_MANAGEMENT.PUBLIC.DRIVER_LOCATIONS AS

SELECT DRIVER_ID, TRIP_ID, POINT_TIME, POINT_GEOM FROM VEHICLE_ROUTING_SIMULATOR.DATA.DRIVER_LOCATIONS;

In [ ]:
CREATE OR REPLACE TABLE FLEET_MANAGEMENT.PUBLIC.TRIPS AS

SELECT TRIP_ID, PICKUP_LOCATION ORIGIN, DROPOFF_LOCATION DESTINATION,

PICKUP_TIME

FROM VEHICLE_ROUTING_SIMULATOR.DATA.NY_TAXI_ROUTE_PLANS;

SELECT * FROM FLEET_MANAGEMENT.PUBLIC.TRIPS LIMIT 5;

### CURATE WITH DRIVER SPEED

In [ ]:
CREATE OR REPLACE TABLE FLEET_MANAGEMENT.ANALYTICS.DRIVER_LOCATIONS AS

WITH lagged AS (
    SELECT
        trip_id,
        point_time,
        point_geom,
        LAG(point_time) OVER (PARTITION BY trip_id ORDER BY point_time) AS prev_time,
        TO_GEOGRAPHY(LAG(ST_ASWKT(point_geom)) OVER (PARTITION BY trip_id ORDER BY point_time)) AS prev_geom
    FROM FLEET_MANAGEMENT.PUBLIC.DRIVER_LOCATIONS
)
SELECT
    trip_id,
    point_time AS curr_time,
    point_geom,
    prev_time,
    ST_DISTANCE(point_geom, prev_geom) AS metres,
    DATEDIFF('second', prev_time, point_time) AS seconds,
    ( ST_DISTANCE(point_geom, prev_geom)
      / NULLIF(DATEDIFF('second', prev_time, point_time), 0)
    ) * 3.6 AS kmh
FROM lagged
WHERE prev_geom IS NOT NULL
  AND DATEDIFF('second', prev_time, point_time) > 0
  AND (ST_DISTANCE(point_geom, prev_geom) / NULLIF(DATEDIFF('second', prev_time, point_time), 0)) * 3.6 <= 200; -- Add this line

### ENRICH WITH POINTS OF INTEREST

In [ ]:
CREATE OR REPLACE WAREHOUSE NYTAXI_PROCESS_DATA_WH WITH WAREHOUSE_SIZE='X-Large';

USE WAREHOUSE NYTAXI_PROCESS_DATA_WH

In [ ]:
CREATE OR REPLACE TRANSIENT TABLE FLEET_MANAGEMENT.PUBLIC.TRIPS_WITH_ORIGINS_POI AS

WITH TripPlacesDistances AS (
    SELECT
        A.*,
        B.NAME AS NEAREST_POI,
        ST_DISTANCE(A.ORIGIN, B.GEOMETRY) AS DISTANCE,
        ROW_NUMBER() OVER (PARTITION BY A.TRIP_ID ORDER BY ST_DISTANCE(A.ORIGIN, B.GEOMETRY)) as rn
    FROM
        FLEET_MANAGEMENT.PUBLIC.TRIPS AS A
    LEFT JOIN
        VEHICLE_ROUTING_SIMULATOR.DATA.PLACES AS B
    ON
        ST_DWITHIN(A.ORIGIN, B.GEOMETRY, 100)
)
SELECT
    TRIP_ID, -- Assuming TRIP_ID is a unique identifier for your trips
    -- Select all columns from TRIPS (A.*) and any other relevant columns
    -- You might want to explicitly list columns from A here, e.g.,
    -- TRIP_ID, TRIP_START_TIME, TRIP_END_TIME, ORIGIN, DESTINATION, etc.
    NEAREST_POI ORIGIN_NEAREST_POI
FROM
    TripPlacesDistances
WHERE
    rn = 1;

In [ ]:
CREATE OR REPLACE  TRANSIENT TABLE FLEET_MANAGEMENT.PUBLIC.TRIPS_WITH_DESTINATION_POI AS

WITH TripPlacesDistances AS (
    SELECT
        A.*,
        B.NAME AS NEAREST_POI,
        ST_DISTANCE(A.ORIGIN, B.GEOMETRY) AS DISTANCE,
        ROW_NUMBER() OVER (PARTITION BY A.TRIP_ID ORDER BY ST_DISTANCE(A.DESTINATION, B.GEOMETRY)) as rn
    FROM
        FLEET_MANAGEMENT.PUBLIC.TRIPS AS A
    LEFT JOIN
        VEHICLE_ROUTING_SIMULATOR.DATA.PLACES AS B
    ON
        ST_DWITHIN(A.DESTINATION, B.GEOMETRY, 100)
)
SELECT
    TRIP_ID, -- Assuming TRIP_ID is a unique identifier for your trips
    -- Select all columns from TRIPS (A.*) and any other relevant columns
    -- You might want to explicitly list columns from A here, e.g.,
    -- TRIP_ID, TRIP_START_TIME, TRIP_END_TIME, ORIGIN, DESTINATION, etc.
    NEAREST_POI DESTINATION_NEAREST_POI
FROM
    TripPlacesDistances
WHERE
    rn = 1;

    

In [ ]:
CREATE OR REPLACE TRANSIENT TABLE FLEET_MANAGEMENT.PUBLIC.NEW_YORK_ADDRESSES AS

SELECT *, REGEXP_REPLACE(

CONCAT(COALESCE(UNIT,''),
' ', 
COALESCE(NUMBER,''), 
' ', COALESCE(STREET,''),
' ', COALESCE(POSTAL_CITY,''),
' ', COALESCE(BOROUGH_CODE,''),
' ', COALESCE(POSTCODE,''),
''),'\\s+', ' ' ) ADDRESS
FROM VEHICLE_ROUTING_SIMULATOR.DATA.NEW_YORK_ADDRESSES;

In [ ]:
CREATE OR REPLACE TRANSIENT TABLE FLEET_MANAGEMENT.PUBLIC.TRIPS_WITH_ORIGIN_ADDRESSES AS

WITH TripPlacesDistances AS (
    SELECT
        A.*,
        B.STREET,
        B.POSTCODE,
        B.NUMBER,
        B.UNIT,
        B.POSTAL_CITY,
        B.BOROUGH_NAME,
        B.ADDRESS,
        ST_DISTANCE(A.ORIGIN, B.GEOMETRY) AS DISTANCE,
        ROW_NUMBER() OVER (PARTITION BY A.TRIP_ID ORDER BY ST_DISTANCE(A.ORIGIN, B.GEOMETRY)) as rn
    FROM
        FLEET_MANAGEMENT.PUBLIC.TRIPS AS A
    LEFT JOIN
        FLEET_MANAGEMENT.PUBLIC.NEW_YORK_ADDRESSES AS B
    ON
        ST_DWITHIN(A.ORIGIN, B.GEOMETRY, 100)
)
SELECT
    TRIP_ID,
    STREET ORIGIN_STREET,
    POSTCODE ORIGIN_POSTCODE,
    NUMBER ORIGIN_NUMBER,
    UNIT ORIGIN_UNIT,
    POSTAL_CITY ORIGIN_POSTAL_CITY,
    BOROUGH_NAME ORIGIN_BOROUGH_NAME,
    ADDRESS ORIGIN_ADDRESS
FROM
    TripPlacesDistances
WHERE
    rn = 1;

In [ ]:
CREATE OR REPLACE TRANSIENT TABLE FLEET_MANAGEMENT.PUBLIC.TRIPS_WITH_DESTINATION_ADDRESSES AS

WITH TripPlacesDistances AS (
    SELECT
        A.*,
        B.ADDRESS,
        B.STREET,
        B.POSTCODE,
        B.NUMBER,
        B.UNIT,
        B.POSTAL_CITY,
        B.BOROUGH_NAME,
        ST_DISTANCE(A.ORIGIN, B.GEOMETRY) AS DISTANCE,
        ROW_NUMBER() OVER (PARTITION BY A.TRIP_ID ORDER BY ST_DISTANCE(A.DESTINATION, B.GEOMETRY)) as rn
    FROM
        FLEET_MANAGEMENT.PUBLIC.TRIPS AS A
    LEFT JOIN
        FLEET_MANAGEMENT.PUBLIC.NEW_YORK_ADDRESSES AS B
    ON
        ST_DWITHIN(A.ORIGIN, B.GEOMETRY, 100)
)
SELECT
    TRIP_ID,
    STREET DESTINATION_STREET,
    POSTCODE DESTINATION_POSTCODE,
    NUMBER DESTINATION_NUMBER,
    UNIT DESTINATION_UNIT,
    POSTAL_CITY DESTINATION_POSTAL_CITY,
    BOROUGH_NAME DESTINATION_BOROUGH_NAME,
    ADDRESS DESTINATION_ADDRESS
FROM
    TripPlacesDistances
WHERE
    rn = 1;


In [ ]:
select * from FLEET_MANAGEMENT.PUBLIC.TRIPS_WITH_DESTINATION_ADDRESSES limit 100

In [ ]:
alter warehouse NYTAXI_PROCESS_DATA_WH suspend;
USE WAREHOUSE DEFAULT_WH;

In [ ]:
CREATE OR REPLACE TABLE FLEET_MANAGEMENT.ANALYTICS.TRIPS AS 


SELECT
    -- Select all columns from all tables, or specify the ones you need
    A.*,
    B.* EXCLUDE (TRIP_ID), -- Exclude TRIP_ID from B to avoid duplicate columns in final output
    C.* EXCLUDE (TRIP_ID), -- Exclude TRIP_ID from C
    D.* EXCLUDE (TRIP_ID), -- Exclude TRIP_ID from D
    E.* EXCLUDE (TRIP_ID)  -- Exclude TRIP_ID from E
FROM
    FLEET_MANAGEMENT.PUBLIC.TRIPS A
INNER JOIN
    FLEET_MANAGEMENT.PUBLIC.TRIPS_WITH_DESTINATION_ADDRESSES B
    ON A.TRIP_ID = B.TRIP_ID -- Explicitly join on TRIP_ID
INNER JOIN
    FLEET_MANAGEMENT.PUBLIC.TRIPS_WITH_ORIGIN_ADDRESSES C
    ON A.TRIP_ID = C.TRIP_ID -- Explicitly join on TRIP_ID
INNER JOIN
    FLEET_MANAGEMENT.PUBLIC.TRIPS_WITH_DESTINATION_POI D
    ON A.TRIP_ID = D.TRIP_ID -- Explicitly join on TRIP_ID
INNER JOIN
    FLEET_MANAGEMENT.PUBLIC.TRIPS_WITH_ORIGINS_POI E
    ON A.TRIP_ID = E.TRIP_ID -- Explicitly join on TRIP_ID
--LIMIT 10;

### Create a route name

In [ ]:
CREATE OR REPLACE TABLE FLEET_MANAGEMENT.ANALYTICS.ROUTE_NAMES AS 

SELECT TRIP_ID, CONCAT(
COALESCE(ORIGIN_ADDRESS,''),
'(',
COALESCE(ORIGIN_NEAREST_POI,''),
')',
'-',
COALESCE(DESTINATION_ADDRESS,''),
'(',
COALESCE(DESTINATION_NEAREST_POI,''),
')') TRIP_NAME

from FLEET_MANAGEMENT.ANALYTICS.TRIPS;

SELECT * FROM ROUTE_NAMES LIMIT 10;

In [ ]:
SELECT * FROM FLEET_MANAGEMENT.ANALYTICS.TRIPS LIMIT 10;

In [ ]:
SELECT * FROM FLEET_MANAGEMENT.ANALYTICS.TRIPS LIMIT 10

### ASSIGN ROUTES
Normally this would take apprx 10 mins to load - so for this exercise, you will just load a sample

In [ ]:
create TABLE if not exists FLEET_MANAGEMENT.ANALYTICS.TRIP_ROUTE_PLAN_SAMPLE (
	TRIP_ID VARCHAR(36),
	ORIGIN GEOGRAPHY,
	DESTINATION GEOGRAPHY,
	PICKUP_TIME TIMESTAMP_NTZ(9),
	DESTINATION_STREET VARCHAR(16777216),
	DESTINATION_POSTCODE VARCHAR(16777216),
	DESTINATION_NUMBER VARCHAR(16777216),
	DESTINATION_UNIT VARCHAR(16777216),
	DESTINATION_POSTAL_CITY VARCHAR(16777216),
	DESTINATION_BOROUGH_NAME VARCHAR(16777216),
	DESTINATION_ADDRESS VARCHAR(16777216),
	ORIGIN_STREET VARCHAR(16777216),
	ORIGIN_POSTCODE VARCHAR(16777216),
	ORIGIN_NUMBER VARCHAR(16777216),
	ORIGIN_UNIT VARCHAR(16777216),
	ORIGIN_POSTAL_CITY VARCHAR(16777216),
	ORIGIN_BOROUGH_NAME VARCHAR(16777216),
	ORIGIN_ADDRESS VARCHAR(16777216),
	DESTINATION_NEAREST_POI VARCHAR(16777216),
	ORIGIN_NEAREST_POI VARCHAR(16777216),
    ROUTE VARIANT
);

## Below creates directions for 404K taxi trips in New York.  This takes 10 minutes.

In [ ]:
INSERT INTO FLEET_MANAGEMENT.ANALYTICS.TRIP_ROUTE_PLAN_SAMPLE

SELECT * FROM (
SELECT
    TRIPS.*, -- Selects all columns from FLEET_MANAGEMENT.ANALYTICS.TRIPS
    OPEN_ROUTE_SERVICE_NEW_YORK.CORE.DIRECTIONS(
        'driving-car',
        ST_ASGEOJSON(TRIPS.ORIGIN):coordinates,
        ST_ASGEOJSON(TRIPS.DESTINATION):coordinates
    ) AS ROUTE
FROM FLEET_MANAGEMENT.ANALYTICS.TRIPS AS TRIPS
ORDER BY TRIPS.TRIP_ID ) sample (1000 rows)


## DRIVER ASSIGNMENT

In [ ]:
---in reality this will be using the route optimisation service.

CREATE OR REPLACE TABLE FLEET_MANAGEMENT.ANALYTICS.TRIPS_ASSIGNED_TO_DRIVERS AS


select *, 
TO_GEOGRAPHY(ROUTE:features[0]:geometry) GEOMETRY 


from (


SELECT * FROM FLEET_MANAGEMENT.ANALYTICS.TRIP_ROUTE_PLAN a

NATURAL JOIN

(SELECT DRIVER_ID, TRIP_ID FROM VEHICLE_ROUTING_SIMULATOR.DATA.NY_TAXI_ROUTE_PLANS) b );


;



select * from FLEET_MANAGEMENT.ANALYTICS.TRIPS_ASSIGNED_TO_DRIVERS limit 10;

In [ ]:
CREATE OR REPLACE TABLE FLEET_MANAGEMENT.ANALYTICS.TRIP_SUMMARY AS 

SELECT 
TRIP_ID, 
MAX(CURR_TIME) ACTUAL_DROPOFF_TIME, 
MIN(CURR_TIME) ACTUAL_PICKUP_TIME, 
AVG(KMH) AVERAGE_KMH,
MIN(KMH) MIN_KMH,
MAX(KMH) MAX_KMH, 
TIMESTAMPDIFF('seconds',MIN(CURR_TIME),MAX(CURR_TIME)) ACTUAL_DURATION 

FROM 

FLEET_MANAGEMENT.ANALYTICS.DRIVER_LOCATIONS 

GROUP BY TRIP_ID;

SELECT * FROM FLEET_MANAGEMENT.ANALYTICS.TRIP_SUMMARY sample (500 rows);